In [1]:
import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from string import punctuation
from sklearn.ensemble import RandomForestClassifier
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import wordpunct_tokenize

In [2]:
stopwords_json = {"en":["a","a's","able","about","above","according","accordingly","across","actually","after","afterwards","again","against","ain't","all","allow","allows","almost","alone","along","already","also","although","always","am","among","amongst","an","and","another","any","anybody","anyhow","anyone","anything","anyway","anyways","anywhere","apart","appear","appreciate","appropriate","are","aren't","around","as","aside","ask","asking","associated","at","available","away","awfully","b","be","became","because","become","becomes","becoming","been","before","beforehand","behind","being","believe","below","beside","besides","best","better","between","beyond","both","brief","but","by","c","c'mon","c's","came","can","can't","cannot","cant","cause","causes","certain","certainly","changes","clearly","co","com","come","comes","concerning","consequently","consider","considering","contain","containing","contains","corresponding","could","couldn't","course","currently","d","definitely","described","despite","did","didn't","different","do","does","doesn't","doing","don't","done","down","downwards","during","e","each","edu","eg","eight","either","else","elsewhere","enough","entirely","especially","et","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","exactly","example","except","f","far","few","fifth","first","five","followed","following","follows","for","former","formerly","forth","four","from","further","furthermore","g","get","gets","getting","given","gives","go","goes","going","gone","got","gotten","greetings","h","had","hadn't","happens","hardly","has","hasn't","have","haven't","having","he","he's","hello","help","hence","her","here","here's","hereafter","hereby","herein","hereupon","hers","herself","hi","him","himself","his","hither","hopefully","how","howbeit","however","i","i'd","i'll","i'm","i've","ie","if","ignored","immediate","in","inasmuch","inc","indeed","indicate","indicated","indicates","inner","insofar","instead","into","inward","is","isn't","it","it'd","it'll","it's","its","itself","j","just","k","keep","keeps","kept","know","known","knows","l","last","lately","later","latter","latterly","least","less","lest","let","let's","like","liked","likely","little","look","looking","looks","ltd","m","mainly","many","may","maybe","me","mean","meanwhile","merely","might","more","moreover","most","mostly","much","must","my","myself","n","name","namely","nd","near","nearly","necessary","need","needs","neither","never","nevertheless","new","next","nine","no","nobody","non","none","noone","nor","normally","not","nothing","novel","now","nowhere","o","obviously","of","off","often","oh","ok","okay","old","on","once","one","ones","only","onto","or","other","others","otherwise","ought","our","ours","ourselves","out","outside","over","overall","own","p","particular","particularly","per","perhaps","placed","please","plus","possible","presumably","probably","provides","q","que","quite","qv","r","rather","rd","re","really","reasonably","regarding","regardless","regards","relatively","respectively","right","s","said","same","saw","say","saying","says","second","secondly","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sensible","sent","serious","seriously","seven","several","shall","she","should","shouldn't","since","six","so","some","somebody","somehow","someone","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specified","specify","specifying","still","sub","such","sup","sure","t","t's","take","taken","tell","tends","th","than","thank","thanks","thanx","that","that's","thats","the","their","theirs","them","themselves","then","thence","there","there's","thereafter","thereby","therefore","therein","theres","thereupon","these","they","they'd","they'll","they're","they've","think","third","this","thorough","thoroughly","those","though","three","through","throughout","thru","thus","to","together","too","took","toward","towards","tried","tries","truly","try","trying","twice","two","u","un","under","unfortunately","unless","unlikely","until","unto","up","upon","us","use","used","useful","uses","using","usually","uucp","v","value","various","very","via","viz","vs","w","want","wants","was","wasn't","way","we","we'd","we'll","we're","we've","welcome","well","went","were","weren't","what","what's","whatever","when","whence","whenever","where","where's","whereafter","whereas","whereby","wherein","whereupon","wherever","whether","which","while","whither","who","who's","whoever","whole","whom","whose","why","will","willing","wish","with","within","without","won't","wonder","would","wouldn't","x","y","yes","yet","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves","z","zero"]}


In [3]:
punc = set(punctuation)
porter = PorterStemmer()
stopwords_json_en = set(stopwords_json['en'])
stopwords_nltk_en = set(stopwords.words('english'))
stopwords_punct = set(punctuation)
spe = ["''","``"]
spe = set(spe)
# Combine the stopwords. Its a lot longer so I'm not printing it out...
stop_pun = set.union(stopwords_json_en, stopwords_nltk_en, stopwords_punct,spe)
count_vect = CountVectorizer()

In [43]:
train=pd.read_csv('wikipedia-detox-250-line-data.tsv.txt',delimiter="\t",encoding='cp1252',names=['Intent','Sentence'],skiprows=1)
test=pd.read_csv('wikipedia-detox-250-line-test.tsv.txt',delimiter="\t",encoding='cp1252',names=['Intent','Sentence'])

In [44]:
train.head()

,Intent,Sentence
0,1,"==RUDE== Dude, you are rude upload that carl..."
1,1,== OK! == IM GOING TO VANDALIZE WILD ONES W...
2,1,"Stop trolling, zapatancas, calling me a lia..."
3,1,==You're cool== You seem like a really cool...
4,1,::::: Why are you threatening me? I'm not bei...


In [45]:
# with open('wikipedia-detox-250-line-data.tsv.txt',encoding='cp1252') as f:
#     for line in f:
#         print(line)


In [40]:
# train=pd.read_csv('enron_train.txt',delimiter="\t",header=None, names=["Intent", "Sentence"])
# test=pd.read_csv('enron_test.txt',delimiter="\t",header=None, names=["Intent", "Sentence"])

In [56]:
train.Intent.value_counts()

1    157
0     89
Name: Intent, dtype: int64

In [57]:
def list_to_str(x):
    y=str(x)
    y1=y.replace('[','')
    y2=y1.replace(']','')
    y3=y2.replace(",","")
    y4=y3.replace("'","")
    return y4

In [58]:
def do_preprocessing(x):
    x=x.lower()
    x = nltk.word_tokenize(x)
    for word in x:
        if word in stop_pun:
            x.remove(word)
        if type(word)==type(5) or type(word)==type(5.0):
            x.remove(word)
    x=list_to_str(x)
    return x
    

In [59]:

def create_train_and_test_data(train,test):
    y_train=train['Intent']
    x_train=train.drop('Intent',axis=1)
    y_test=test['Intent']
    x_test=test.drop('Intent',axis=1)
    
    x_train['Sentence']=x_train[['Sentence']].applymap(lambda t :do_preprocessing(t))
    train_data=x_train['Sentence']
    x_test['Sentence']=x_test[['Sentence']].applymap(lambda t :do_preprocessing(t))
    test_data=x_test['Sentence']
    
#     count_vect = CountVectorizer()
    train_data=count_vect.fit_transform(train_data)
    test_data=count_vect.transform(test_data)
    
    tfidf_transformer=TfidfTransformer()
    train_data=tfidf_transformer.fit_transform(train_data)
    test_data=tfidf_transformer.fit_transform(test_data)
    return train_data,y_train,test_data,y_test
    

In [60]:
x_train,y_train,x_test,y_test=create_train_and_test_data(train,test)
type(x_train),type(x_test)

(scipy.sparse.csr.csr_matrix, scipy.sparse.csr.csr_matrix)

In [61]:
x_train.shape,x_test.shape

((246, 2726), (18, 2726))

In [62]:
model=RandomForestClassifier(n_estimators=500,min_samples_leaf=5)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
accuracy=accuracy_score(y_pred,y_test)*100
accuracy

50.0

In [75]:
from sklearn.neural_network import MLPClassifier
model=MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
model.fit(x_train,y_train)

y_pred=model.predict(x_test)

accuracy=accuracy_score(y_pred,y_test)*100
accuracy

66.66666666666666

In [66]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(x_train,y_train)

y_pred=model.predict(x_test)

accuracy=accuracy_score(y_pred,y_test)*100
accuracy

55.55555555555556

In [70]:
from sklearn.svm import SVC
model=SVC()
model.fit(x_train,y_train)

y_pred=model.predict(x_test)

accuracy=accuracy_score(y_pred,y_test)*100
accuracy

/home/niki/.local/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


50.0

In [76]:
def get_sentiment(x):
    x=[do_preprocessing(x)]
#     count_vect = CountVectorizer()
#     count_vect.fit_transform(train['Sentence'])
    x_vect=count_vect.transform(x)
    
    tfidf_transformer=TfidfTransformer()
    x_tfidf=tfidf_transformer.fit_transform(x_vect)
    pred=model.predict(x_tfidf)
    return pred[0]
print(get_sentiment('look forward not happy seeing next week'))

1
